In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
import glob
#%matplotlib widget
import xarray as xr

In [ ]:
%matplotlib widget

In [ ]:
import pointCollection as pc

In [ ]:
from netCDF4 import Dataset

In [ ]:
%matplotlib widget

In [ ]:
nc_file='/att/nobackup/project/icesat-2/ATL14_processing/rel001/south/AA/ATL15_AA_0310_01km_001_01.nc'

In [ ]:
with xr.open_dataset(nc_file,group='/dhdt_lag1') as fh:
    dzdt_bar=pc.grid.data().from_dict({'x':np.array(fh['x']),'y':np.array(fh['y']),'time':np.array(fh['time']),'z':np.mean(fh['dhdt'][2:,:,:], axis=0)})
    sigma_dzdt=pc.grid.data().from_dict({'x':np.array(fh['x']),'y':np.array(fh['y']),'time':np.array(fh['time']),'z':np.std(fh['dhdt'][2:,:,:], axis=0)})

In [ ]:
plt.figure(); 
sigma_dzdt.show(clim=[0, 2], cmap='magma', interpolation='nearest')
plt.colorbar(shrink=0.25, orientation='horizontal', pad=0.02,  label='$\sigma_{dh/dt}$, $m\ yr^{-1}$')
plt.gca().set_facecolor('gray')


In [ ]:
#fh= xr.open_dataset(nc_file,group='/delta_h')
dh=pc.grid.data().from_dict({'x':np.array(fh['x']),'y':np.array(fh['y']),'z':fh['delta_h'],'time', fh['time'])})

In [ ]:
np.array(fh['time'])

In [ ]:
dir(fh['delta_h'])

In [ ]:
plt.figure()
plt.imshow(np.std(dhdt[2:-1], axis=0), extent=extent, origin='lower', clim=[0, 5], cmap='magma', interpolation='nearest')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.colorbar(shrink=0.25, orientation='horizontal', pad=0.02,  label='$\sigma_{dh/dt}$, $m\ yr^{-1}$')
plt.gca().set_facecolor('gray')



In [ ]:
fig=plt.figure()
plt.imshow(np.mean(dhdt[2:-1], axis=0), extent=extent, origin='lower', clim=[-3, 3], cmap='Spectral', interpolation='nearest')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
hb=plt.colorbar(shrink=0.25, orientation='horizontal', pad=0.02,  label='$dh/dt$, $m\ yr^{-1}$')
plt.gca().set_facecolor('gray')



In [ ]:
area=np.array(dsx['cell_area'])
dvdt=np.zeros(dhdt.shape[0])+np.NaN

t=np.float64(dsx['time_lag1']-np.datetime64('2018-01-01'))/24/3600/365.25/1.e9+2018.0
for ind in range(dhdt.shape[0]):
    dvdt[ind]=np.nansum(area*dhdt[ind,:,:])
plt.figure(); plt.plot(t, dvdt/1.e9,'.')
for ii in range(len(dvdt)):
    plt.plot(t[ii]+np.array([-1, 1])*0.125, (dvdt[ii]+np.zeros(2))/1.e9,'k')
plt.gca().set_xticks([ 2019, 2019.5, 2020, 2020.5])
plt.gca().set_xlabel('year')
plt.gca().set_ylabel('volume rate, $km^3\ yr^{-1}$')

In [ ]:
np.mean(dvdt)/1.e9

## Map of dhdt

In [ ]:
t_bin=3
"%2.2f" % (dsx["time.year"][t_bin]+(dsx["time.dayofyear"][t_bin])/365)
#f'dhdt, {dsx["time"][t_bin].years()} to {dsx["time"][t_bin+1]}'

In [ ]:
plt.figure();

t_bin=2
plt.imshow(dhdt[t_bin,:,:], origin='lower', extent=extent, cmap='Spectral', clim=[-2, 2], interpolation='nearest')
plt.colorbar(label=f'dhdt, %2.2f to %2.2f' % (dsx["time.year"][t_bin]+(dsx["time.dayofyear"][t_bin])/365,
                                             dsx["time.year"][t_bin+4]+(dsx["time.dayofyear"][t_bin+4])/365))

In [ ]:
plt.figure(); 
dhdt=np.array(xr.open_dataset(thefile, group='height_change')['dhdt_lag1'])

plt.imshow(np.sqrt(np.mean(dhdt[2:-1,:,:]**2, axis=0)), origin='lower', extent=extent, clim=[0, 1])

In [ ]:
map_ax=plt.gca()
XY0=[np.round(np.array(map_ax.get_xlim()).mean()/4.e4)*40 , 
    np.round(np.array(map_ax.get_ylim()).mean()/4.e4)*40]
print(XY0)
thefile=glob.glob(f"/home/besmith4/shared/ATL14_test/south/001/tiles/*/problems/E{int(XY0[0])}_N{int(XY0[1])}.h5")[0]
print(thefile)




In [ ]:
dsx=xr.open_dataset(thefile, group='height_change')
dh=np.array(dsx['delta_h'])
xx=dsx['x']
yy=dsx['y']
tt=dsx['time']
extent=[np.min(xx)+500, np.max(xx)+500, np.min(yy)+500, np.max(yy)+500]

In [ ]:
tt=dsx['time.year']+dsx['time.dayofyear']/365

In [ ]:
np.array(tt)

In [ ]:
dh.shape

In [ ]:
plt.figure(); 
plt.imshow(dh[8,:,:]-dh[4,:,:], extent=extent, cmap='Spectral', clim=[-2, 2], origin='lower')
plt.colorbar(label='%2.2f to %2.2f' % (tt[4], tt[8]), shrink=0.5, extend='both')


In [ ]:
plt.figure(); 
plt.imshow(dh[6,:,:]-dh[2,:,:], extent=extent, cmap='Spectral', clim=[-2, 2], origin='lower')
plt.colorbar(label='%2.2f to %2.2f' % (tt[6], tt[2]), shrink=0.5, extend='both')


In [ ]:

fig=plt.figure(figsize=[6, 7.5])
hax=fig.subplots(3,2).ravel()
for count, ii in enumerate(range(2, 8)):
    hi=hax[count].imshow(dh[ii+1,:,:]-dh[ii,:,:], extent=extent, cmap='Spectral', clim=[-0.5, 0.5], origin='lower')
    hax[count].set_title('%2.2f to %2.2f' % (tt[ii], tt[ii+1]), fontsize=10)
    hax[count].set_xticks([])
    hax[count].set_yticks([])
#hax[count+1].remove()
plt.colorbar(hi, ax=hax, label='quarterly $\delta h$, m', orientation='horizontal', shrink=0.25,pad=0.025, extend='both')
#plt.tight_layout()
    

In [ ]:
tt[-1]-tt[2]

In [ ]:
plt.figure(figsize=[6,7])
plt.imshow((dh[-1,:,:]-dh[2,:,:])/float(tt[-1]-tt[2]), extent=extent, cmap='Spectral', clim=[-0.75, 0.75], origin='lower')
plt.colorbar(label='dh/dt, 2019.25 to 2020.75, m/yr', pad=0.025, shrink=0.5, extend='both', orientation='horizontal')
plt.gca().set_xticks([])
plt.gca().set_yticks([])


## Map of dhdt and tile centers

In [ ]:
plt.figure(); 
plt.imshow(dhdt[3,:,:], origin='lower', extent=extent, cmap='Spectral', clim=[-1, 1])
syms={'centers':'s','edges':'^','corners':'x'}
for key, temp in xy_tile.items():
    xy=np.c_[list(temp.keys())]
    plt.plot(xy[:,0]*1000, xy[:,1]*1000, marker=syms[key], linestyle='')

In [ ]:
Dtiff=pc.grid.data().from_geotif('/home/besmith4/shared/ATL14_test/north/001/tiles/temp.tif')

In [ ]:
plt.figure(); Dtiff.show(band=0)

In [ ]:
plt.figure(); plt.plot(Dtiff.x, Dtiff.z[130,:, 0], label='tiff')
plt.plot(xx, temp[0, 130,:], 'o', label='xarray')

In [ ]:
temp.shape

In [ ]:
! h5ls /home/besmith4/shared/ATL14_test/south/001/tiles/edges/E-960_N280.h5

In [ ]:
dz0=pc.grid.data().from_h5('/home/besmith4/shared/ATL14_test/south/001/tiles/edges/E-960_N280.h5', group='dz')

In [ ]:
dz1=pc.grid.data().from_h5('/home/besmith4/shared/ATL14_test/south/001/tiles/edges/problems/E-960_N280.h5', group='dz')

In [ ]:
fig=plt.figure(); 
fig.add_subplot(122)
plt.imshow(dz0.dz[:,:,4]-dz0.dz[:,:,0], extent=dz0.extent, origin='lower', clim=[-1, 1], cmap='Spectral')
fig.add_subplot(121)
plt.imshow(dz1.dz[:,:,4]-dz1.dz[:,:,0], extent=dz0.extent, origin='lower' , clim=[-1, 1], cmap='Spectral')
